In [105]:
import tensorflow as tf
import tensorflowvisu
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
import math
print("Tensorflow version " + tf.__version__)
tf.set_random_seed(0)

mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)


Tensorflow version 1.3.0
Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [112]:
pkeep = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 28, 28, 1])

W1 = tf.Variable(tf.truncated_normal([28*28, 200] ,stddev=0.1))
B1 = tf.Variable(tf.zeros([200]))

W2 = tf.Variable(tf.truncated_normal([200, 100] ,stddev=0.1))
B2 = tf.Variable(tf.zeros([100]))

W3 = tf.Variable(tf.truncated_normal([100, 50], stddev=0.1))
B3 = tf.Variable(tf.zeros([50]))

W4 = tf.Variable(tf.truncated_normal([50, 30], stddev=0.1))
B4 = tf.Variable(tf.zeros([30]))

W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1))
B5 = tf.Variable(tf.zeros([10]))

XX = tf.reshape(X, [-1, 28*28])

Y1 = tf.nn.relu(tf.matmul(XX, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
Y2d = tf.nn.dropout(Y2, pkeep)

Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
Y3d = tf.nn.dropout(Y3, pkeep)

Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
Y4d = tf.nn.dropout(Y4, pkeep)

Ylogits = tf.matmul(Y4d, W5) + B5
Y = tf.nn.softmax(Ylogits)

Y_ = tf.placeholder(tf.float32, [None, 10])

lr = tf.placeholder(tf.float32)

In [113]:
#cross_entropy = -tf.reduce_mean(Y_ * tf.log(Y)) * 1000.0  # normalized for batches of 100 images,
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100


correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.train.AdamOptimizer(lr)
train_step= optimizer.minimize(cross_entropy)

In [114]:
test_accuracy = []

In [118]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

def training_step(i):
    batch_X, batch_Y = mnist.train.next_batch(100)
    
    max_learning_rate = 0.003
    min_learning_rate = 0.0001
    decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    train_data = {X: batch_X, Y_: batch_Y, lr: learning_rate, pkeep: 0.75}
        
    if i%100 == 0:
        a = sess.run(accuracy, feed_dict={X: mnist.test.images, Y_: mnist.test.labels, pkeep: 1})
        print(str(i) + ' test accuracy: ' + str(a))
        
        
    sess.run(train_step, feed_dict=train_data)

In [119]:
for i in range(10000+1): training_step(i)

0 test accuracy: 0.1141
100 test accuracy: 0.8657
200 test accuracy: 0.9149
300 test accuracy: 0.9386
400 test accuracy: 0.9459
500 test accuracy: 0.9486
600 test accuracy: 0.9502
700 test accuracy: 0.9572
800 test accuracy: 0.9614
900 test accuracy: 0.9603
1000 test accuracy: 0.9627
1100 test accuracy: 0.9651
1200 test accuracy: 0.9639
1300 test accuracy: 0.9673
1400 test accuracy: 0.9679
1500 test accuracy: 0.9699
1600 test accuracy: 0.9713
1700 test accuracy: 0.971
1800 test accuracy: 0.9712
1900 test accuracy: 0.9678
2000 test accuracy: 0.9724
2100 test accuracy: 0.9733
2200 test accuracy: 0.9731
2300 test accuracy: 0.9745
2400 test accuracy: 0.9735
2500 test accuracy: 0.9744
2600 test accuracy: 0.9749
2700 test accuracy: 0.9741
2800 test accuracy: 0.9753
2900 test accuracy: 0.9749
3000 test accuracy: 0.9745
3100 test accuracy: 0.9757
3200 test accuracy: 0.9765
3300 test accuracy: 0.9762
3400 test accuracy: 0.9774
3500 test accuracy: 0.9778
3600 test accuracy: 0.9769
3700 test accu